In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd

In [3]:
year = 2022
month = 2
taxi_type = 'yellow'

input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"output/{taxi_type}/{year:04d}-{month:02d}.parquet"

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(input_file)

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [8]:
y_pred

array([18.52778307, 23.06578208, 33.68635854, ..., 11.89045938,
       15.10268128,  9.46059157])

In [9]:
# save above y_pred array to pandas dataframe
df_y_pred = pd.DataFrame(data=y_pred, columns=['predicted'])

In [10]:
df_y_pred.head()

,predicted
0,18.527783
1,23.065782
2,33.686359
3,23.757436
4,21.492904


In [11]:
# What's the standard deviation of the predicted duration for this df?
print('std dev of predicted duration:', df_y_pred['predicted'].std())

std dev of predicted duration: 5.281404481465351


In [12]:
# What's the mean of the predicted duration for this df?
print('Mean of predicted duration:', df_y_pred['predicted'].mean())

Mean of predicted duration: 12.513422116701408


In [13]:
df_y_pred.describe()

,predicted
count,2.918187e+06
mean,1.251342e+01
std,5.281404e+00
min,-7.485578e+00
25%,9.945231e+00
50%,1.066465e+01
75%,1.213280e+01
max,8.013667e+01


###  Preparing the output

In [14]:
# let's create an artificial ride_id column
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [15]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,ride_id
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,...,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333,2022/02_0
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,...,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000,2022/02_1
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,...,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000,2022/02_2
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,...,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333,2022/02_3
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,...,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667,2022/02_4


In [16]:
# create empty results df
df_result = pd.DataFrame()

In [17]:
df_result['ride_id'] = df['ride_id']

In [18]:
df_result.head()

,ride_id
0,2022/02_0
1,2022/02_1
2,2022/02_2
3,2022/02_3
4,2022/02_4


In [19]:
# df_result['tpep_pickup_datetime'] = df['tpep_pickup_datetime']
# df_result['PULocationID'] = df['PULocationID']
# df_result['DOLocationID'] = df['DOLocationID']
# df_result['actual_duration'] = df['duration']
df_result['predictions'] = df_y_pred
# df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']

In [20]:
df_result.head()

,ride_id,predictions
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


In [21]:
# create output and yellow dir, if it does not exist
# !mkdir output
# !mkdir output/yellow

In [22]:
# !ls output

In [23]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [24]:
!ls -lh output/yellow/

total 58M
-rw-rw-r-- 1 ranga ranga 58M Jun 26 00:41 2022-02.parquet
